In [1]:
import pandas as pd
# import sys
from glob import glob
import os
import os.path as osp
import random
import numpy as np
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Resize, ToTensor, CenterCrop, Normalize, GaussianBlur, RandomRotation, ColorJitter, RandomHorizontalFlip, RandomResizedCrop
from sklearn.metrics import f1_score
#난수 시드 설정
# torch.manual_seed(1234)
# np.random.seed(1234)
# random.seed(1234)

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))
print(f"CUDA: {torch.cuda.is_available()}")
print(f"PyTorch Version : ", torch.__version__)
print(f"torch Version: {torch.__version__}")

device:[cuda:0].
CUDA: True
PyTorch Version :  1.7.1
torch Version: 1.7.1


In [3]:
# torch.backends.cudnn.deterministic= True
# torch.backends.cudnn.benchmark = False

In [4]:
data_dir = '/opt/ml/input/data/train'
img_dir = f'{data_dir}/images'
df_path = f'{data_dir}/train.csv'
df = pd.read_csv(df_path)

In [5]:
# N = 5

# fig, ax = plt.subplots(7,N)
# fig.set_figheight(30)
# fig.set_figwidth(15)

# for j, n in enumerate(np.random.randint(0,len(df),size = N)):
#     path = "/opt/ml/input/data/train/images/" + df["path"][n]
#     i = 0
#     for im_name in os.listdir(path):
#         if not im_name.startswith("."):
#             im = Image.open(path + '/' + im_name)

#             ax[i][j].set_title(im_name)
#             ax[i][j].imshow(im)
#             ax[i][j].axis("off")
#             i += 1
        
# plt.show()


In [6]:
# im_avg = np.zeros((512,384,3),dtype = np.float32)
# for n in range(len(df)):
#     path = "/opt/ml/input/data/train/images/" + df["path"][n]
#     for im_name in os.listdir(path):
#         if not im_name.startswith("."):
#             im_avg += np.array(Image.open(path + '/' + im_name),dtype = np.float32)/225
            
# im_avg = im_avg/(len(df)*7)
# fig, ax = plt.subplots(1,1, figsize = (6, 4))
# ax.imshow(im_avg)

# plt.show()

In [7]:
# import matplotlib.patches as patches
# x_1, y_1 = 100, 100
# x_2, y_2 = 300, 400

# fig, ax = plt.subplots(1,1, figsize = (12, 8))

# ax.imshow(im_avg)

# # Create a Rectangle patch
# rect = patches.Rectangle(
#     xy       = (80, 50),
#     width    = 220,
#     height   = 320,
#     linewidth= 1,
#     edgecolor= 'r',
#     facecolor= 'none'
# )

# # Add the patch to the Axes
# ax.add_patch(rect)

# plt.show()

In [8]:
from torchvision.transforms.functional import crop

def crop800(image):
    return crop(image, 80, 50, 370, 300)

# x_1, y_1 = 100, 100
# x_2, y_2 = 300, 400
# fig, ax = plt.subplots(1,1, figsize = (12, 8))

# ax.imshow(crop800(im_avg))

# data_transforms = {
#     'images': transforms.Compose([transforms.ToTensor(),
#                                   transforms.Lambda(crop800),
#                                   transforms.Resize((400, 400))])}

In [9]:

class ImageTransform():
    def __init__(self, resize, mean, std,phase=('train','value')):
        transformations = {}
        self.data_transform = {
            'train' : transforms.Compose([
#                 transforms.RandomResizedCrop(resize, scale = (0.5,1.0)), #데이터 확장
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation([-2, +2]),
                transforms.GaussianBlur(51, (0.1, 2.0)),
                transforms.ColorJitter(brightness=0.9, saturation=0.5, hue=0.5),  # todo : param
                transforms.ToTensor(),
                transforms.Lambda(crop800),
                transforms.Normalize(mean, std),
            ]),
            'val' : transforms.Compose([
                transforms.Resize(resize),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
            ])
        }
    def __call__(self, img, phase='train'):
        return self.data_transform[phase](img)

In [20]:
# # 화상 전처리 확인

# #1. 화상 읽기
# image_file_path = '/opt/ml/input/data/train/images/000001_female_Asian_45/mask1.jpg'
# img = Image.open(image_file_path) #[높이][너비][색RGB]
# #2. 원본 화상 표시
# plt.imshow(img)
# plt.show()

# #3. 화상 전처리 및 처리된 화상의 표시
size = 300
mean = (0.560, 0.524, 0.501)
std = (0.233, 0.243, 0.245)
transform = ImageTransform(size, mean, std)
# img_transformed = transform(img, phase="train") #torch.Size([3, 300, 300])
# # # (색상, 높이, 너비)를 (높이, 너비, 색상)으로 변환하고 0-1 로 값을 제한하여 표시
# img_transformed = img_transformed.numpy().transpose((1, 2, 0))
# img_transformed = np.clip(img_transformed, 0, 1)
# plt.imshow(img_transformed)
# plt.show()

In [21]:
class MaskLabels:
    mask = 0
    incorrect = 1
    normal = 2

class GenderLabels:
    male = 0
    female = 1

class AgeGroup:
    map_label = lambda x: 0 if int(x) < 30 else 1 if int(x) < 60 else 2

In [22]:
class HymenopterDataset(data.Dataset):
    num_classes = 3 * 2 * 3

    _file_names = {
        "mask1.jpg": MaskLabels.mask,
        "mask2.jpg": MaskLabels.mask,
        "mask3.jpg": MaskLabels.mask,
        "mask4.jpg": MaskLabels.mask,
        "mask5.jpg": MaskLabels.mask,
        "incorrect_mask.jpg": MaskLabels.incorrect,
        "normal.jpg": MaskLabels.normal
    }

    image_paths = []
    mask_labels = []
    gender_labels = []
    age_labels = []

    def __init__(self, img_dir, transform=None):
        """
        MaskBaseDataset을 initialize 합니다.

        Args:
            img_dir: 학습 이미지 폴더의 root directory 입니다.
            transform: Augmentation을 하는 함수입니다.
        """
        self.img_dir = img_dir
#         self.mean = mean
#         self.std = std
        self.transform = transform
        self.setup()

    def set_transform(self, transform):
        """
        transform 함수를 설정하는 함수입니다.
        """
        self.transform = transform
        
    def setup(self):
        """
        image의 경로와 각 이미지들의 label을 계산하여 저장해두는 함수입니다.
        """
        profiles = os.listdir(self.img_dir)
        for profile in profiles:
            for file_name, label in self._file_names.items():
                img_path = os.path.join(self.img_dir, profile, file_name)  # (resized_data, 000004_male_Asian_54, mask1.jpg)
                if os.path.exists(img_path):
                    self.image_paths.append(img_path)
                    self.mask_labels.append(label)

                    id, gender, race, age = profile.split("_")
                    gender_label = getattr(GenderLabels, gender)
                    age_label = AgeGroup.map_label(age)

                    self.gender_labels.append(gender_label)
                    self.age_labels.append(age_label)

    def __getitem__(self, index):
        """
        데이터를 불러오는 함수입니다. 
        데이터셋 class에 데이터 정보가 저장되어 있고, index를 통해 해당 위치에 있는 데이터 정보를 불러옵니다.
        
        Args:
            index: 불러올 데이터의 인덱스값입니다.
        """
        # 이미지를 불러옵니다.
        image_path = self.image_paths[index]
        image = Image.open(image_path)
        
        # 레이블을 불러옵니다.
        mask_label = self.mask_labels[index]
        gender_label = self.gender_labels[index]
        age_label = self.age_labels[index]
        multi_class_label = mask_label * 6 + gender_label * 3 + age_label
        
        # 이미지를 Augmentation 시킵니다.
        image_transform = self.transform(image)
        return image_transform, multi_class_label
        
        if not self.split_labels:
            multi_class_label = mask_label * 6 + gender_label * 3 + age_label
            return image_transform, multi_class_label
        else:
            return image_transform, (mask_label, gender_label, age_label)

    def __len__(self):
        return len(self.image_paths)

In [23]:
train_img_dir = '/opt/ml/input/data/train/images'
val_img_dir = '/opt/ml/input/data/eval/images'

In [24]:
# train dataset과 validation dataset을 8:2 비율로 나눕니다.
dataset =HymenopterDataset(img_dir=train_img_dir)

n_val = int(len(dataset) * 0.1)
n_train = len(dataset) - n_val

train_dataset, val_dataset = torch.utils.data.random_split(dataset, 
                                                           [n_train, n_val])
train_transform =ImageTransform(size, mean, std, phase = 'train')
val_transform = ImageTransform(size, mean, std, phase = 'val')
# val_dataset = transform(transform['val'])

train_dataset = HymenopterDataset(img_dir = train_img_dir, transform = train_transform)

val_dataset = HymenopterDataset(img_dir = val_img_dir, transform = val_transform)

In [25]:
# 동작 확인
index = 0
print(train_dataset.__getitem__(index)[0].size()) #Chanel:3 width: 300 height :300
print(train_dataset.__getitem__(index)[1]) # mask, incorrect_mask, normal

torch.Size([3, 370, 300])
3


In [ ]:
#미니 배치 크기 지정:
batch_size = 32

#데이터 로더 작성
train_dataloader = data.DataLoader(
    train_dataset, batch_size = batch_size, shuffle = True, num_workers = 2, pin_memory = True)
val_dataloader = data.DataLoader(
    val_dataset, batch_size = batch_size, shuffle = False)

# 사전형 변수에 정리
dataloaders_dict = {"train" : train_dataloader, "val" : val_dataloader}

#동작 확인
batch_iterator = iter(dataloaders_dict["train"])
inputs, labels = next(
    batch_iterator)
print(inputs.size())
print(labels)

In [ ]:
#학습된 VGG-16 모델 로드
# VGG-16 모델의 인스턴스 생성
use_pretrained = True #학습된 파라미터 사용
net = models.vgg16(pretrained = use_pretrained)

#VGG-16 마지막 출력층의 출력 유닛을 18개의 클래스로 변경
net.classifier[6] = nn.Linear(in_features = 1000, out_features = 18)
#훈련 모드로 설정
net.train()
print('네트워크 설정 완료 : 학습된 가중치를 읽어들여 훈련 모드로 설정했습니다.')
print(net)

In [ ]:
criterion = nn.CrossEntropyLoss()

# 전이학습에서 학습시킬 파라미터를 params_to_update 변수에 저장
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []


#학습시킬 파라미터명
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight",
                        "classifier.0.biase", "classifier.3.weight",
                        "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

#학습시킬 파라미터 외에는 경사를 계산하지 않고 변하지 않도록 설정
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("params_to_update_1에 저장: ", name)
    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("params_to_update_2에 저장: ", name)        
    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("params_to_update_3에 저장: ", name) 
    else:
        param.requires_grad =False
        print("경사 계산 없음. 학습하지 않음: ", name)

#최적화 방법 설정
optimizer = optim.SGD([
    {'params' : params_to_update_1, 'lr' :1e-4},
    {'params' : params_to_update_2, 'lr' :5e-4},
    {'params' : params_to_update_3, 'lr' :1e-3}
], momentum = 0.9, weight_decay = 0.9)

In [ ]:
# !wandb login

In [ ]:
# import wandb

# config = dict(learning_rate =0.01,
#               momentum = 0.9,
#               architecture = "VGG16",
#               dataset_id = "MarkBaseDataset",)
# run = wandb.init(
#     project = "singon",
#     name = "my_test_10",
#     notes = "base_line_test",
#     tags = ["Holiday"],
#     config = config,
# )

In [23]:
import time
#네트워크를 GPU로
dtype = torch.float
ltype = torch.long
net.to(device)
# torch.backends.cudnn.benchmark = True
num_epochs = 1

valid_early_stop = 0
valid_best_loss = float('inf')
EARLY_STOPPING_EPOCH = 5
since = time.time()

final_train_loss = []
final_train_acc = []
final_valid_loss = []
final_valid_acc = []


for e in range(num_epochs) :
    print(f' ====================== epoch %d ======================' % (e+1) )
    train_loss_list = []
    train_acc_list = []
    epoch_f1 = 0
    n_iter = 0
    # train
    net.train()
    for i, (images, targets) in enumerate(train_dataloader) : 
        optimizer.zero_grad()

        images = images.to(device, dtype)
        targets = targets.to(device, ltype)

        scores = net(images)
        _, preds = torch.max(scores,1)

        loss = criterion(scores, targets)
        loss.backward()
        optimizer.step()
        
        correct = sum(targets == preds).cpu()
        acc=(correct/32 * 100)

        train_loss_list.append(loss)
        train_acc_list.append(acc)

        if i % 20 == 0 :
            print(f'Iteration %3.d | Train Loss  %.4f | Classifier Accuracy %2.2f' % (i, loss, acc))
        del images, targets
        
    torch.cuda.empty_cache()
    
    train_mean_loss = np.mean(train_loss_list, dtype="float64")
    train_mean_acc = np.mean(train_acc_list, dtype="float64")

#     wandb.log({"train_average accuracy" : train_mean_acc})
#     wandb.log({"train_average_loss" : train_mean_loss})    

    
    final_train_loss.append(train_mean_loss)
    final_train_acc.append(train_mean_acc)

    epoch_time = time.time() - since
    since = time.time()

    print('')
    print(f'[Summary] Elapsed time : %.0f m %.0f s' % (epoch_time // 60, epoch_time % 60))
    print(f'Train Loss Mean %.4f | Accuracy %2.2f ' % (train_mean_loss, train_mean_acc) )

    # validation 
    valid_loss_list = []
    valid_acc_list = []

    net.eval()
    for i, (images, targets) in enumerate(val_dataloader) : 
        
        images = images.to(device=device, dtype=dtype)
        targets = targets.to(device=device, dtype=ltype)
        optimizer.zero_grad()
        
        with torch.no_grad():
            scores = net(images)
            _, preds = torch.max(scores,1)
            loss = criterion(scores, targets)
            loss.backward()
            optimizer.step()
        
        epoch_f1 += f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
        n_iter += 1

        correct = sum(targets == preds).cpu()
        acc=(correct/32 * 100)

        valid_loss_list.append(loss)
        valid_acc_list.append(acc)
        
        if (i+1) % 20 == 0 :
            print(f'Iteration %3.d | validation Loss  %.4f | Class Acc(validation) %2.2f' % (i+1, loss, acc))
        del images, targets
        
    torch.cuda.empty_cache()

    val_mean_loss = np.mean(valid_loss_list, dtype="float64")
    val_mean_acc = np.mean(valid_acc_list, dtype="float64")


    final_valid_loss.append(val_mean_loss)
    final_valid_acc.append(val_mean_acc)
 

    print(f'average valid loss %.4f | average valid accuracy %2.2f ' % (val_mean_loss, val_mean_acc) )
    print('')
    
    epoch_f1 = epoch_f1/n_iter
    print(f"{epoch_f1:.4f}")

    if val_mean_loss < valid_best_loss:
        valid_best_loss = val_mean_loss
        valid_early_stop = 0
        # new best model save (valid 기준)
        best_model = net
#         wandb.log({"average validation accuracy" : val_mean_acc})
#         wandb.log({"average validation loss" : val_mean_loss})

    else:
        # early stopping    
        valid_early_stop += 1
        if valid_early_stop >= EARLY_STOPPING_EPOCH:
            print("EARLY STOPPING!!")
            break

 ====================== epoch 1 ======================
Iteration   0 | Train Loss  2.9310 | Classifier Accuracy 3.12
Iteration  20 | Train Loss  2.3175 | Classifier Accuracy 28.12


KeyboardInterrupt: 

In [ ]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [ ]:

# meta 데이터와 이미지 경로를 불러옵니다.
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize, CenterCrop

test_dir = '/opt/ml/input/data/eval'
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    CenterCrop((300, 200)),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

# transform = get_transforms(mean=mean, std=std)

# dataset = MaskBaseDataset(
#     img_dir=test_dir
# )

# dataset.dataset.set_transform(transform['val'])

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
net = best_model
net.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device=device, dtype=dtype)
        images = images.to(device)
        pred = net(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, '0828_submission_VGG16.csv'), index=False)
print('test inference is done!')